<a href="https://colab.research.google.com/github/KimiaGoodarzi/MyProjects/blob/main/EyesOpenClosed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Library Imports**: The essential PyTorch libraries and utilities are imported.



In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import numpy as np


**File Upload:** Google Colab's file upload utility is being used to upload files directly into the runtime's environment. The code block uploads our dataset in the form of a zip file named "train 2.zip".

In [4]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))


Saving train 2.zip to train 2.zip
User uploaded file "train 2.zip" with length 25429753 bytes


**Zip File Extraction:** The uploaded dataset is extracted, then the images inside the zip file will be available in our directory.

In [5]:
import zipfile
import os

zip_ref = zipfile.ZipFile('train 2.zip', 'r')
zip_ref.extractall()
zip_ref.close()


**Data Loading and Preprocessing**

Images are transformed by:
Resizing to 64x64 pixels.
Converting them to tensor format
Normalizing pixel values between -0.5 and 0.5 for better convergence during training.

Dataset Loading: The ImageFolder utility from torchvision is used to load the images.
Dataset Splitting: The dataset is split into training and testing datasets. **80%** of the data is used for training, and **20%** is used for testing.

 For efficient loading of data in batches during the training process, we create PyTorch DataLoader objects.

In [6]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader

transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


dataset = datasets.ImageFolder('train 2', transform=transform)


train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)


**CNNmodel Class: **
The network structure is defined with two convolutional layers followed by two fully connected layers:
conv1: A 2D convolutional layer with 3 input channels, 32 output channels, a kernel size of **3x3**, and a stride of **1**.
conv2: A 2D convolutional layer with 32 input channels (from conv1), 64 output channels, a kernel size of **3x3**, and a stride of **1**.
fc1: A fully connected layer that takes the output, then outputs 128 elements.
fc2: Another fully connected (linear) layer that takes the 128 elements from the previous layer, outputs 2 elements.

In [13]:
import torch.nn as nn
import torch.nn.functional as F

class CNNmodel(nn.Module):
    def __init__(self):
        super(CNNmodel, self).__init__()


        self.conv1 = nn.Conv2d(3, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)

        self.fc1 = nn.Linear(14*14*64, 128)
        self.fc2 = nn.Linear(128, 2)
    def forward(self, x):

        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)

        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)


        x = torch.flatten(x, 1)


        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)

        return F.log_softmax(x, dim=1)

model = CNNmodel()


In [14]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [15]:

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()


        if batch_idx % 10 == 0:
            print(f'Epoch: {epoch+1}/{num_epochs}, Batch: {batch_idx}/{len(train_loader)}, Loss: {loss.item()}')


Epoch: 1/10, Batch: 0/100, Loss: 0.6971750855445862
Epoch: 1/10, Batch: 10/100, Loss: 0.2882293164730072
Epoch: 1/10, Batch: 20/100, Loss: 0.22569438815116882
Epoch: 1/10, Batch: 30/100, Loss: 0.07349244505167007
Epoch: 1/10, Batch: 40/100, Loss: 0.056704457849264145
Epoch: 1/10, Batch: 50/100, Loss: 0.017251694574952126
Epoch: 1/10, Batch: 60/100, Loss: 0.025376975536346436
Epoch: 1/10, Batch: 70/100, Loss: 0.17431190609931946
Epoch: 1/10, Batch: 80/100, Loss: 0.06582852452993393
Epoch: 1/10, Batch: 90/100, Loss: 0.009061189368367195
Epoch: 2/10, Batch: 0/100, Loss: 0.014648564159870148
Epoch: 2/10, Batch: 10/100, Loss: 0.002554418984800577
Epoch: 2/10, Batch: 20/100, Loss: 0.007007355336099863
Epoch: 2/10, Batch: 30/100, Loss: 0.012217767536640167
Epoch: 2/10, Batch: 40/100, Loss: 0.00661088339984417
Epoch: 2/10, Batch: 50/100, Loss: 0.03262922167778015
Epoch: 2/10, Batch: 60/100, Loss: 0.01086620707064867
Epoch: 2/10, Batch: 70/100, Loss: 0.002093823626637459
Epoch: 2/10, Batch: 80/

In [16]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data, target in test_loader:
        output = model(data)
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

print(f'Accuracy on the test set: {100 * correct / total}%')


Accuracy on the test set: 100.0%
